In [0]:
from time import sleep
import requests
import re
import json
import datetime
import gzip


dbutils.widgets.text('reportdate', '')
dbutils.widgets.text('marketplace', '')

ReportDate = dbutils.widgets.get('reportdate')
ReportDate = ReportDate[0:4]+ReportDate[5:7]+ReportDate[8:10]  #yyyyMMdd
marketplace = dbutils.widgets.get('marketplace')

# LwA Dev User 
C_KEY = 'amzn1.applicat..............8d82777b0b3'
C_SECRET_KEY = 'd12827396da1.................d07e2a89a767c4'
C_REFRESH_TOKEN = 'Atzr|IwEBILSZA-9uzSdXoA...................tHWH7r4UJWzMLp--2n5R7r3bXFg'

# ProfileIds can be retrieved from request: https://advertising-api-eu.amazon.com/v2/profiles
if marketplace == 'DE':
    ProfileId = '21692...4659'
    
elif  marketplace == 'IT':
    ProfileId = '2155...123194'
    
elif marketplace == 'FR':
    ProfileId = '11836...6966079'
    
elif marketplace == 'NL':
     ProfileId = '37179...632014'
        
elif marketplace == 'ES':
     ProfileId = '12197...53006'
        
elif marketplace == 'UK':
     ProfileId = '3411...77547'   

In [0]:
# Get Access Token
def get_access_token():
    global refresh_token
    url = "https://api.amazon.co.uk/auth/o2/token"

    body = {
            "grant_type":"refresh_token",
            "client_id": C_KEY,
            "client_secret": C_SECRET_KEY,
            "refresh_token":C_REFRESH_TOKEN
           }
 
    response = requests.post(url = url, data = body)
    return response.json()['access_token']

C_ACCESS_TOKEN = get_access_token()

In [0]:
# Generate Report and Get ReportId. In metrics, sku is only for Seller Accounts. If it is Vendor Account delete sku. 
url = "https://advertising-api-eu.amazon.com/v2/sp/productAds/report"

headers = {
            'Authorization'                   : 'Bearer ' + C_ACCESS_TOKEN,
            'Amazon-Advertising-API-ClientId' :  C_KEY,
            'Amazon-Advertising-API-Scope'    :  ProfileId,
            'Content-Type'                    : 'application/json'
            }

body = {
            'reportDate'  : ReportDate,
            'stateFilter' : 'enabled,paused,archived',
            'metrics': 'adGroupId,adGroupName,asin,sku,campaignId,campaignName,campaignStatus,clicks,cost,currency,impressions,attributedConversions14d,attributedConversions14dSameSKU,attributedConversions1d,attributedConversions1dSameSKU,attributedConversions30d,attributedConversions30dSameSKU,attributedConversions7d,attributedConversions7dSameSKU,attributedSales14d,attributedSales14dSameSKU,attributedSales1d,attributedSales1dSameSKU,attributedSales30d,attributedSales30dSameSKU,attributedSales7d,attributedSales7dSameSKU,attributedUnitsOrdered14d,attributedUnitsOrdered14dSameSKU,attributedUnitsOrdered1d,attributedUnitsOrdered1dSameSKU,attributedUnitsOrdered30d,attributedUnitsOrdered30dSameSKU,attributedUnitsOrdered7d,attributedUnitsOrdered7dSameSKU,campaignBudget,campaignBudgetType'
          }


body = json.dumps(body)
generate = requests.post(url = url, headers = headers, data = body)
reportId = generate.json()['reportId']

In [0]:
# Download the Report - Report generation can take as long as 15 minutes
sleep(900)
url = f"https://advertising-api-eu.amazon.com/v2/reports/{reportId}/download"

headers = {
            'Authorization'                   : 'Bearer ' + C_ACCESS_TOKEN,
            'Amazon-Advertising-API-ClientId' :  C_KEY,
            'Amazon-Advertising-API-Scope'    :  ProfileId,
            'Content-Type'                    : 'application/json'
            }


report = requests.get(url = url, headers = headers)
zipfile = report.content
file = (gzip.decompress(zipfile).decode('utf-8'))

In [0]:
# Connect to Blob Storage and Mounting. 
# sas token can be taken from the container settings > shared access tokens.
# Permissions should be given Read, Write, Delete ..(sp=racwdli&st) while generating sas token. Give expire date range widely so that dont get auth error. 
containerName = "amzn-api-sp"
storageAccountName = "stgaadvertising"
sas = "sp=racwdli&st=2022-08-01T14:42:20Z&se=2029-08-01.......hksYzi4vfbzD80K%2BAPjwtm0%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/amzn-api-sp")
except:
    pass
    
try:
    dbutils.fs.mount( source = "wasbs://{}@{}.blob.core.windows.net".format(containerName,storageAccountName),
                      mount_point = "/mnt/amzn-api-sp",
                      extra_configs = {config : sas}
                    )
except:
    pass

/mnt/amzn-api-sp has been unmounted.

In [0]:
# Put file to the blob container
dbutils.fs.put('dbfs:/mnt/amzn-api-sp/' + ReportDate + '__' + marketplace + '.json', file)

Wrote 13745376 bytes.
Out[25]: True